# Criar navegador e importar dados

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Abrindo navegardor
servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)

tabela_produtos = pd.read_excel('Buscas.xlsx')
display(tabela_produtos)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.58M/6.58M [00:00<00:00, 10.8MB/s]


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 128 gb,mini watch usado,3000,4000
1,gtx 1660,rtx zotac galax,850,1850


# Definir funções de busca nas plataformas


In [14]:
def busca_google(driver, produto, termos_banidos, preco_minimo, preco_maximo):

    # Tratar valores da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produtos = produto.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    # Entrar no Google, pesquisar o produto e acessar Shopping
    driver.get('https://www.google.com.br/')
    
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    
    elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if 'Shopping' in item.text:
            item.click()
            break

    # Pegar lista de resultados da busca no Google Shopping
    lista_resultados = driver.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')   
    
    # Para cada resultado, verificar se corresponde às condições
    lista_ofertas = []
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nome = nome.lower()
        
        # Se há termos banidos no nome 
        ban_termos=False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                ban_termos = True
        
        # Se o produto da tabela corresponde ao nome da oferta
        tem_produtos = True
        for palavra in lista_termos_produtos:
            if palavra not in nome:
                tem_produtos = False

        # Pegando os preços e transformando em float
        if not ban_termos and tem_produtos: # verificando o nome
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                # verificando se o preco ta dentro do minimo e maximo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
            
    return lista_ofertas

def busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # Tratar valores da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produtos = produto.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    # Entrar no Buscapé 
    driver.get('https://www.buscape.com.br/')
    
    # Pesquisar o produto
    driver.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__HG105').send_keys(produto, Keys.ENTER)
    
    time.sleep(7)
    
    # Pegar a lista de resultado
    elementos = driver.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
    
    
    lista_ofertas = []
    for resultado in elementos:
        try:
            time.sleep(3)
            nome = resultado.find_element(By.CLASS_NAME, 'Text_DesktopLabelSAtLarge__xHZpN').text
            nome = nome.lower()
            
           
    # Verificar termos banidos
   
            ban_termos=False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    ban_termos = True

            # Se o produto da tabela corresponde ao nome da oferta
            tem_produtos = True
            for palavra in lista_termos_produtos:
                if palavra not in nome:
                    tem_produtos = False

            # Pegando os preços e transformando em float
            if not ban_termos and tem_produtos: # verificando o nome
                preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__XS_Au').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                # verificando se o preco ta dentro do minimo e maximo
                if preco_minimo <= preco <= preco_maximo:
                    link = resultado.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))

        except:
            pass
    
    return lista_ofertas
    
    

# Integrando funções ao banco de dados e buscando nas plataformas

In [15]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, 'Nome']
    termos_banidos = tabela_produtos.loc[linha, 'Termos banidos']
    preco_minimo = tabela_produtos.loc[linha, 'Preço mínimo']
    preco_maximo = tabela_produtos.loc[linha, 'Preço máximo']
    lista_ofertas_google = busca_google(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google:
        tabela_google = pd.DataFrame(lista_ofertas_google, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google)
    else:
        tabela_google = None
    lista_ofertas_buscape = busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:    
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None

display(tabela_ofertas)    

,Produto,Preço,Link
0,iphone 12 128gb black seminovo,3500.00,https://www.google.com.br/url?url=https://rn.o...
1,iphone 12 128gb - azul - estou zerado,3518.10,https://www.google.com.br/url?url=https://www....
2,iphone 12 128gb - preto - estou zerado,3518.10,https://www.google.com.br/url?url=https://www....
3,"(seminovos) iphone 12 apple branco, 128gb",3699.00,https://www.google.com.br/url?url=https://www....
4,apple iphone 11 128gb preto,3533.07,https://www.google.com.br/url?url=https://www....
5,apple iphone 12 pro (128gb) silver,3714.77,https://www.google.com.br/url?url=https://mobi...
6,apple iphone 12 pro (128gb) dourado,3714.77,https://www.google.com.br/url?url=https://mobi...
7,"(seminovos) iphone 12 apple preto, 128gb",3699.00,https://www.google.com.br/url?url=https://www....
8,apple iphone 11 (128 gb) branco,3499.00,https://www.google.com.br/url?url=https://www....
9,iphone 12 pro 128gb vitrine,3600.00,https://www.google.com.br/url?url=https://www....


# Salvar planilha de preços

In [16]:
# Exportando para excel
tabela_ofertas = tabela_ofertas.reset_index(drop=True)
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

# Enviar e-mail de alerta de preços

In [17]:
import win32com.client as win32

In [21]:
# Verificando se existe oferta
if len(tabela_ofertas.index) > 0:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'mister_jorge@hotmail.com'
    mail.Subject = 'Produtos Encontrados'

    mail.HTMLBody =f'''
    <p>Jorge,</p>
    <p>Foram encontradas algumas ofertas que podem ser de seu interesse. Segue a tabela com mais detalhes!</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Caso haja algum erro, entrar em contato!</p>
    <p>Atenciosamente,</p>
    '''
    
    mail.Send()
    
driver.quit()